In [1]:
import sklearn 
import os 
import numpy as np
from google.colab import drive
root = "/content/drive"
drive.mount(root)
os.chdir("/content/drive/My Drive/ps_hw_3")

Mounted at /content/drive


In [2]:
run = np.loadtxt("jogging.csv", delimiter=",")
walk = np.loadtxt("walk.csv", delimiter=",")
stay = np.loadtxt("still.csv", delimiter=",")
drive = np.loadtxt("driving.csv", delimiter=",")
climb = np.loadtxt("climbing.csv",delimiter=",")

In [21]:
stay_data = np.hstack((stay, np.ones(stay.shape[0]).reshape((stay.shape[0] ,1)) ))
drive_data = np.hstack((drive, np.ones(drive.shape[0]).reshape((drive.shape[0] ,1)) * 2 ))
walk_data = np.hstack((walk, np.ones(walk.shape[0]).reshape((walk.shape[0] ,1)) * 3))
run_data = np.hstack((run, np.ones(run.shape[0]).reshape((run.shape[0] ,1)) * 4))
climb_data = np.hstack((climb, np.ones(climb.shape[0]).reshape((climb.shape[0] ,1)) * 5))

In [22]:
stay_data_1 = stay_data[:int(stay_data.shape[0]/30)*30,:]
drive_data_1 = drive_data[:int(drive_data.shape[0]/30)*30,:]
walk_data_1 = walk_data[:int(walk_data.shape[0]/30)*30,:]
run_data_1 = run_data[:int(run_data.shape[0]/30)*30,:]
climb_data_1 = climb_data[:int(climb_data.shape[0]/30)*30,:]

In [31]:
A1 = np.zeros((int(stay_data.shape[0]/30),3))
A2 = np.zeros((int(drive_data.shape[0]/30),3))
A3 = np.zeros((int(walk_data.shape[0]/30),3))
A4 = np.zeros((int(run_data.shape[0]/30),3))
A5 = np.zeros((int(climb_data.shape[0]/30),3))

In [34]:
for i in range(A1.shape[0]):
    A1[i,:] = np.mean(stay_data_1[i * 30 : (i + 1) * 30, :3], axis = 0)
for i in range(A2.shape[0]):
    A2[i,:] = np.mean(drive_data_1[i * 30 : (i + 1) * 30, :3], axis = 0)
for i in range(A3.shape[0]):
    A3[i,:] = np.mean(walk_data_1[i * 30 : (i + 1) * 30, :3], axis = 0)
for i in range(A4.shape[0]):
    A4[i,:] = np.mean(run_data_1[i * 30 : (i + 1) * 30, :3], axis = 0)
for i in range(A5.shape[0]):
    A5[i,:] = np.mean(climb_data_1[i * 30 : (i + 1) * 30, :3], axis = 0)

In [42]:
B1 = np.hstack((A1.copy(), np.ones(A1.shape[0], dtype = np.int8).reshape(A1.shape[0],1) * 1))
B2 = np.hstack((A2.copy(), np.ones(A2.shape[0], dtype = np.int8).reshape(A2.shape[0],1) * 2))
B3 = np.hstack((A3.copy(), np.ones(A3.shape[0], dtype = np.int8).reshape(A3.shape[0],1) * 3))
B4 = np.hstack((A4.copy(), np.ones(A4.shape[0], dtype = np.int8).reshape(A4.shape[0],1) * 4))
B5 = np.hstack((A5.copy(), np.ones(A5.shape[0], dtype = np.int8).reshape(A5.shape[0],1) * 5))

In [44]:
data = np.vstack((B1,B2,B3,B4,B5))

In [45]:
from sklearn.utils import shuffle
data_1 = shuffle(data, random_state = 42)

In [62]:
import xgboost as xgb
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
X_train, X_test, y_train, y_test = train_test_split(data_1[:, :3], data_1[:,3].astype(np.int8), test_size = 0.2, random_state=1024)
clf = xgb.XGBClassifier(n_jobs=1)
clf.fit(X_train, y_train, early_stopping_rounds=10, eval_metric="mlogloss",
        eval_set=[(X_test, y_test)], verbose = False)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [63]:
pred_y = clf.predict(X_test)

In [64]:
np.sum(pred_y == y_test) / X_test.shape[0]

0.8435374149659864

In [65]:
xgb_model = xgb.XGBClassifier(n_jobs=1)
clf = GridSearchCV(xgb_model,
                   {'max_depth': [2, 4, 6],
                    'n_estimators': [50, 100, 200]}, verbose=1, n_jobs=1)
clf.fit(X_train, y_train)
print(clf.best_score_)
print(clf.best_params_)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


0.8321544356027115
{'max_depth': 4, 'n_estimators': 100}


[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:    5.5s finished


In [69]:
clf = xgb.XGBClassifier(n_jobs=1, max_depth=4, n_estimators=100)
clf.fit(X_train, y_train, early_stopping_rounds=10, eval_metric="mlogloss",
        eval_set=[(X_test, y_test)], verbose = False)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=4,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [70]:
pred_y = clf.predict(X_test)

First one supervised method: Xgboost test-accuracy 0.84

In [71]:
np.sum(pred_y == y_test) / X_test.shape[0]

0.8435374149659864

Start to try logistic regression 

In [72]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=1024, multi_class = "multinomial").fit(X_train, y_train)
pred_y = clf.predict(X_test)
np.sum(pred_y == y_test) / X_test.shape[0]

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.6870748299319728

The second one supervised method: logistic regression test-accuracy 0.68

Start to try kernelized svm
default is "rbf" i.e. gaussian kernel

In [74]:
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(X_train, y_train)
pred_y = clf.predict(X_test)
np.sum(pred_y == y_test) / X_test.shape[0]

0.8435374149659864

Kernel svm give accuary same as xgboost